## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,2021-09-17 14:54:21,-21.2078,-159.7750,73.45,78,40,16.11,scattered clouds
1,1,Fukue,JP,2021-09-17 14:58:10,32.6881,128.8419,74.16,83,100,22.37,overcast clouds
2,2,Vila Velha,BR,2021-09-17 14:58:11,-20.3297,-40.2925,78.75,78,40,12.66,scattered clouds
3,3,Yellowknife,CA,2021-09-17 14:56:05,62.4560,-114.3525,41.52,100,75,10.36,broken clouds
4,4,Rikitea,PF,2021-09-17 14:53:01,-23.1203,-134.9692,67.03,88,100,8.99,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,2021-09-17 14:54:21,-21.2078,-159.7750,73.45,78,40,16.11,scattered clouds
1,1,Fukue,JP,2021-09-17 14:58:10,32.6881,128.8419,74.16,83,100,22.37,overcast clouds
2,2,Vila Velha,BR,2021-09-17 14:58:11,-20.3297,-40.2925,78.75,78,40,12.66,scattered clouds
4,4,Rikitea,PF,2021-09-17 14:53:01,-23.1203,-134.9692,67.03,88,100,8.99,moderate rain
6,6,Cape Town,ZA,2021-09-17 14:52:38,-33.9258,18.4232,72.12,60,0,4.00,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

430


City_ID                430
City                   430
Country                427
Date                   430
Lat                    430
Lng                    430
Max Temp               430
Humidity               430
Cloudiness             430
Wind Speed             430
Current Description    430
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

427


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,2021-09-17 14:54:21,-21.2078,-159.7750,73.45,78,40,16.11,scattered clouds
1,1,Fukue,JP,2021-09-17 14:58:10,32.6881,128.8419,74.16,83,100,22.37,overcast clouds
2,2,Vila Velha,BR,2021-09-17 14:58:11,-20.3297,-40.2925,78.75,78,40,12.66,scattered clouds
4,4,Rikitea,PF,2021-09-17 14:53:01,-23.1203,-134.9692,67.03,88,100,8.99,moderate rain
6,6,Cape Town,ZA,2021-09-17 14:52:38,-33.9258,18.4232,72.12,60,0,4.00,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,73.45,scattered clouds,-21.2078,-159.7750,
1,Fukue,JP,74.16,overcast clouds,32.6881,128.8419,
2,Vila Velha,BR,78.75,scattered clouds,-20.3297,-40.2925,
4,Rikitea,PF,67.03,moderate rain,-23.1203,-134.9692,
6,Cape Town,ZA,72.12,clear sky,-33.9258,18.4232,
10,Road Town,VG,86.04,scattered clouds,18.4167,-64.6167,
12,Hilo,US,74.28,moderate rain,19.7297,-155.0900,
13,Katsuura,JP,75.31,overcast clouds,35.1333,140.3000,
15,Yinchuan,CN,71.96,overcast clouds,38.4681,106.2731,
17,Chuy,UY,63.61,clear sky,-33.6971,-53.4616,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
H

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
H

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vila Velha,BR,78.75,scattered clouds,-20.3297,-40.2925,Quality Suites Vila Velha
10,Road Town,VG,86.04,scattered clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
18,Havelock,US,82.38,mist,34.8791,-76.9013,Sherwood Motel
27,Kapaa,US,75.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
29,Batemans Bay,AU,63.99,moderate rain,-35.7167,150.1833,Batemans Bay Marina Resort
56,Illapel,CL,67.60,scattered clouds,-31.6308,-71.1653,Hotel Londres
61,Hasaki,JP,73.80,overcast clouds,35.7333,140.8333,Hotel Sunrise Choshi
65,Hiriyur,IN,77.77,few clouds,13.9456,76.6192,Royal Inn
87,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
94,Atuona,PF,77.58,few clouds,-9.8000,-139.0333,Villa Enata


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))